In [ ]:
import simpy
import numpy as np
import matplotlib.pyplot as plt
import random


ModuleNotFoundError: No module named 'simpy'

In [ ]:
# Used as a guide: https://simpy.readthedocs.io/en/latest/examples/bank_renege.html

def source(env, time_total, Lambda, counter, waiting_times, mu):

    for i in range(time_total):
                
        c = customer(env, f'Customer{i:02d}', counter, waiting_times, mu)

        env.process(c)

        t = random.expovariate(Lambda)

        yield env.timeout(t)

def customer(env, name, counter, timer, mu):

    arrive = env.now

    with counter.request() as req:

        yield req

        wait = env.now - arrive
        timer.append(wait)

        time_in = random.expovariate(mu)
        yield env.timeout(time_in)


def run_simulation(num_servers, N, Lambda, mu, time_total):
    
    timer = []

    for test in range(N):

        env = simpy.Environment()

        counter = simpy.Resource(env, capacity=num_servers)
 
        env.process(source(env, time_total, Lambda, counter, timer, mu))

        env.run()

    return np.mean(timer)

In [ ]:
mu = 1
Lambda = np.linspace(0.1, 0.99, 20)
time_total = 1000

#amount of repeats
N = 50

#For now only made with one run, not taking averages yet
for servers in [1,2,4]:
 
    collected_times = []

    for value in Lambda:

        result = run_simulation(servers, 1, value, mu, time_total)
        collected_times.append(result)


    plt.plot(Lambda, collected_times, label = f'n = {servers}')

plt.legend()
plt.show()

In [ ]:

# Trying again for different load values
load = np.linspace(0.1, 0.99, 30)

mu = 1
time_total = 1000

#amount of repeats
N = 100

for servers in [1,2,4]:
 
    collected_times = []

    for value in load:

        Lambda = value * servers * mu

        result = run_simulation(servers, N, Lambda, mu, time_total)
        collected_times.append(result)


    plt.plot(load, collected_times,'o', label = f'n = {servers}')

plt.legend()
plt.grid()
plt.yscale('log')
plt.xlabel('load')
plt.ylabel('average wait time')
plt.show()